# Рынок заведений общественного питания Москвы <a class="tocSkip">

Идея проекта - открыть небольшое кафе в Москве. Оно оригинальное — гостей должны обслуживать роботы. Проект многообещающий, но дорогой. Необходимо проанализировать текущее положение ресторанного бизнеса на рынке - насколько актуальна идея с внедрением роботов-официантов и сможет ли данная идея "выстрелить", а также получится ли снискать и сформировать стек постоянных клиентов, учитывая инновационность идеи. 

У нас имеются данные о заведениях общественного питания в Москве. 

Описание данных
Таблица `rest_data`:

* `id` — идентификатор объекта;
* `object_name` — название объекта общественного питания;
* `chain` — сетевой ресторан;
* `object_type` — тип объекта общественного питания;
* `address` — адрес;
* `number` — количество посадочных мест.

## Загрузка данных и подготовка к анализу

In [1]:
!pip install --upgrade plotly

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: plotly in /home/jovyan/.local/lib/python3.7/site-packages (4.14.3)


In [47]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

import plotly.express as px 
from plotly import graph_objects as go

import tensorflow as tf

In [48]:
rest_data = pd.read_csv('/datasets/rest_data.csv')

Посмотрим какие данные содержатся в таблице, вызвав рандомные строки.

In [49]:
display(pd.concat([rest_data.sample(5, random_state=10)]).reset_index(drop=True).style)

,id,object_name,chain,object_type,address,number
0,27352,Столовая гимназии № 1576 корпус 1,нет,столовая,"город Москва, Большая Академическая улица, дом 22А",190
1,80355,Кофейня Авеню,нет,кафе,"город Москва, улица Бориса Галушкина, дом 10",38
2,192985,Кафе «Кулинарная лавка братьев Караваевых»,да,магазин (отдел кулинарии),"город Москва, улица Тимура Фрунзе, дом 11, строение 7",10
3,22648,ТД ГУМ КП Кафе «Фестивальное» БЛИНЫ,нет,кафе,"город Москва, Красная площадь, дом 3",36
4,218404,Кулибин,нет,ресторан,"город Москва, Хорошёвское шоссе, дом 27",40


Видим, что в столбце `object_name` содержатся названия объектов общественного питания, написанные большими и маленькими буквами. Это может приводить к наличию дубликатов, если одно  и то же кафе написано по-разному.

In [50]:
rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
id             15366 non-null int64
object_name    15366 non-null object
chain          15366 non-null object
object_type    15366 non-null object
address        15366 non-null object
number         15366 non-null int64
dtypes: int64(2), object(4)
memory usage: 720.4+ KB


После выгрузки общей информации о таблице видим, что пропущенные значения отсутствуют, в каждом столбце 15 366 заполненных строк. Но также видим, что мы можем оптимизировать работу кода понизив разрядность столбцов `id` и `number`. Также переведем тип данных `object_type` и `chain` в тип `category`.

In [57]:
#Понижение разрядности
rest_data['id'] = pd.to_numeric(rest_data['id'], downcast='signed')
rest_data['number'] = pd.to_numeric(rest_data['number'], downcast='signed')

In [58]:
#Перевод столбца из типа object в category
rest_data['object_type'] = rest_data['object_type'].astype('category')
rest_data['chain'] = rest_data['chain'].astype('category')

Проверим количество дубликатов в данных.

In [53]:
#Количество дубликатов до приведения к нижнему регистру
print(f'Количество дубликатов: {rest_data.duplicated().sum()}')

Количество дубликатов: 0


Дубликатов не обнаружено. Чтобы окончательно в этом убедиться, приведем столбцы `object_name` и `address` к нижнему регистру и проверим на наличие дубликатов.

In [60]:
#Приведение столбцов object_name и address к нижнему регистру
rest_data['object_name'] = rest_data['object_name'].str.lower()
rest_data['address'] = rest_data['address'].str.lower()

In [55]:
print(f'Количество дубликатов после приведения к нижнему регистру: {rest_data.duplicated().sum()}')

Количество дубликатов после приведения к нижнему регистру: 0


После проверки дубликатов также не обнаружено.

## Исследуйте соотношение видов объектов общественного питания по количеству. Постройте график.

Построим график соотношения видов объектов общественного питания по количеству.

In [45]:
object_count = rest_data.groupby('object_type').agg({'object_type':'count'})\
    .rename(columns={'object_type':'count'}).reset_index().sort_values(by='count', ascending=False)

In [46]:
fig = px.bar(object_count, y='object_type', x='count')
#fig.update_xaxes(tickangle=45)
fig.update_layout(
    title='Cоотношение видов объектов общественного питания по количеству',
    xaxis_title="Количество объектов",
    yaxis_title="Тип объекта")
fig.show() 

По графику видим, что наиболее часто встречающийся вид объектов общественного питания - кафе (6099 объектов). Второе и третье место принадлежит столовым и ресторанам - 2587 и 2285 объектов соответственно. Меньше всего объектов в нашей выборке - магазины (отдел кулинарии): всего 273 объекта.

## Исследуйте соотношение сетевых и несетевых заведений по количеству. Постройте график.

Построим график соотношения сетевых и несетевых заведений по количеству.

In [61]:
chain_count = rest_data.groupby('chain').agg({'chain':'count'})\
    .rename(columns={'chain':'count'}).reset_index().sort_values(by='count', ascending=False)

In [84]:
fig = go.Figure()
fig.add_trace(go.Pie(values=chain_count['count'], labels=chain_count['chain'], hole=.6, 
                     title='Cоотношение заведений', titlefont=dict(size=20)))
fig.update_layout(title='Cоотношение сетевых и несетевых заведений по количеству',
                 legend_title = "Сетевое заведение?")
fig.show()

По графику видим, что в наших данных несетевых заведений болье - 80.7% от всего числа заведений или 12 398 объектов, в то время как сетевых 19,3% от общего количества или 2968 объектов.